In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is available and memory growth is set.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Using CPU.")


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50, VGG19, EfficientNetB0, EfficientNetB3
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, InputLayer, Dropout, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import os
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Activation

import datetime
import matplotlib.pyplot as plt
from keras_tuner import RandomSearch
from keras import regularizers
from tensorflow import keras


In [ ]:
def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label


In [ ]:
import tensorflow as tf

train_dir = '/mnt/e/Documents/Course/Deep Learning/Projects/Deep Learning Projects/CancerDetection/Dataset/train'  
val_dir = '/mnt/e/Documents/Course/Deep Learning/Projects/Deep Learning Projects/CancerDetection/Dataset/val' 

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred', 
    label_mode='binary',   
    image_size=(224, 224),  
    batch_size=32, 
    shuffle=True,  
    seed=42,  
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=42,
)

class_names = train_dataset.class_names
print("Class names:", class_names)


In [ ]:
train_dataset = train_dataset.map(augment_image)

In [ ]:
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
def display_random_images(dataset, num_images=16):
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1): 
        for i in range(num_images):
            ax = plt.subplot(4, 4, i + 1)
            random_index = np.random.randint(0, images.shape[0]) 
            plt.imshow(images[random_index].numpy().astype("uint8"))  
            plt.title(class_names[int(labels[random_index].numpy())]) 
            plt.axis("off")

display_random_images(train_dataset, num_images=16)
plt.show()

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", patience = 10, restore_best_weights=True)

In [ ]:
total_layers = len(base_model.layers)
print(total_layers)

In [ ]:
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:30]:
    layer.trainable = False

for layer in base_model.layers[30:]:
    layer.trainable = True

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    BatchNormalization(),  
    Dropout(0.5),  
    Dense(1, activation='sigmoid')  
])

optimizer = Adam(learning_rate = 1e-3)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=500,
    callbacks=[early_stopping]
)

In [ ]:


# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()


In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/mnt/e/Documents/Course/Deep Learning/Projects/Deep Learning Projects/CancerDetection/Dataset/test",
    labels='inferred',
    label_mode='binary',
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=42,
)

In [ ]:
import numpy as np

# Assuming test_dataset is your dataset loaded with image_dataset_from_directory
y_true = []
y_pred = []

# Loop through the dataset and collect true labels (y_true)
for images, labels in test_dataset:
    y_true.extend(labels.numpy())  # Extract true labels (y_true)

# Get predicted probabilities from the model
y_pred_prob = model.predict(test_dataset)

# Convert predicted probabilities to class labels (y_pred)
# For binary classification, we assume a threshold of 0.5
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary labels (0 or 1)

# Convert y_true list to a numpy array
y_true = np.array(y_true)

print(f"True labels: {y_true}")
print(f"Predicted labels: {y_pred}")



In [ ]:
!pip install scikit-learn


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Assuming `y_true` are true labels and `y_pred` are predicted labels
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()
